In [ ]:
using QuantumCollocation
using NamedTrajectories
using TrajectoryIndexingUtils

using CairoMakie
using Distributions
using LinearAlgebra

In [ ]:
# Operators 
const n_levels = 2
at = create(n_levels)
a = annihilate(n_levels)

H_operators = Dict(
        "X" => a + at,
        "Y" => -im * (a - at),
        "Z" => I - 2 * at * a,
)

# Time
T = 50
Δt = 0.2
;

In [ ]:
H_drift = zeros(n_levels^4, n_levels^4)
H_controls = [
    kron_from_dict("XIII", H_operators),
    kron_from_dict("IXII", H_operators),
    kron_from_dict("IIXI", H_operators),
    kron_from_dict("IIIX", H_operators),
    # kron_from_dict("YIII", H_operators),
    # kron_from_dict("IYII", H_operators),
    # kron_from_dict("IIYI", H_operators),
    # kron_from_dict("IIIY", H_operators)
]

X_gate = GATES[:X]
SX_gate = sqrt(GATES[:X])
U_goal = reduce(kron, [X_gate, X_gate, SX_gate, SX_gate])
;

In [ ]:
prob = UnitarySmoothPulseProblem(
    H_drift,
    H_controls,
    U_goal,
    T,
    Δt;
    timesteps_all_equal=true,
    free_time=false,
    hessian_approximation=true,
    pade_order=10,
    R=10.,
    R_dda=5.,
)

In [ ]:
solve!(prob; max_iter=75)

unitary_fidelity(prob)

In [ ]:
function plot_control(prob::QuantumControlProblem)
    # plot(prob.trajectory; comps=[:a, :da, :dda])
    f = Figure(resolution=(800, 600))
    ax1 = Axis(f[1, 1])
    ax2 = Axis(f[2, 1])
    ax3 = Axis(f[3, 1])
    ts = accumulate(+, timesteps(prob.trajectory))
    as = prob.trajectory[:a]
    das = prob.trajectory[:da]
    ddas = prob.trajectory[:dda]
    colors = [:red, :green, :blue, :orange, :purple, :yellow, :black]

    for i in 1:size(as, 1)
        lines!(ax1, ts, as[i, :]; color=colors[i])
        lines!(ax2, ts, das[i, :]; color=colors[i])
        lines!(ax3, ts, ddas[i, :]; color=colors[i])
    end
    return f
end


In [ ]:
plot_control(prob)

In [ ]:
function random_a_guess(traj::NamedTrajectory)
    # Positive (symmetric) upper bounds
    a_bounds = traj.bounds[:a][2]

    a_dists = [Uniform(
        -(a_bounds[i] == Inf ? 1.0 : a_bounds[i]),
        (a_bounds[i] == Inf ? 1.0 : a_bounds[i])
    ) for i = 1:traj.dims[:a]]

    a = hcat([
        zeros(traj.dims[:a]),
        vcat([rand(a_dists[i], 1, T - 2) for i = 1:traj.dims[:a]]...),
        zeros(traj.dims[:a])
    ]...)
    return a
end

In [ ]:
@views function infidelity_robustness(Hₑ::AbstractMatrix, p::QuantumControlProblem)
    Z⃗ = vec(p.trajectory.data)
    Z = p.trajectory
    return InfidelityRobustnessObjective(Hₑ, Z).L(Z⃗, Z)
end

In [ ]:
H_crosstalk = (
    kron_from_dict("ZZII", H_operators)
    + kron_from_dict("IZZI", H_operators)
    + kron_from_dict("IIZZ", H_operators)
    + kron_from_dict("ZIIZ", H_operators)
    + kron_from_dict("ZIII", H_operators)
    + kron_from_dict("IZII", H_operators)
    + kron_from_dict("IIZI", H_operators)
    + kron_from_dict("IIIZ", H_operators)
)

trajectory = copy(prob.trajectory)
# Need random initial conditions to avoid local minima
update!(trajectory, :a, random_a_guess(trajectory))
update!(trajectory, :Ũ⃗, 2 * rand(trajectory.dims[:Ũ⃗], T) .- 1)
parameters = deepcopy(prob.params)

objective = DefaultObjective()
objective += Objective(parameters[:objective_terms][1])
objective += QuadraticRegularizer(:dda, prob.trajectory, 1e-3)
objective += QuadraticRegularizer(:a, prob.trajectory, 1e-6)

update_bound!(trajectory, :a, 1.0)
constraints = trajectory_constraints(trajectory)

ipopt_options = Options()
ipopt_options.hessian_approximation = "limited-memory"

rob_prob_1 = UnitaryRobustnessProblem(
    H_crosstalk,
    trajectory,
    prob.system,
    objective,
    prob.integrators,
    constraints;
    final_fidelity=0.99, 
    verbose=false,
    build_trajectory_constraints=false,
    hessian_approximation=true,
    ipopt_options=ipopt_options
)

In [ ]:
solve!(rob_prob_1; max_iter=25)

(i -> println("Fidelity $i"))(unitary_fidelity(rob_prob_1))
(i -> println("Robustness $i"))(infidelity_robustness(H_crosstalk, rob_prob_1))

In [ ]:
plot_control(rob_prob_1)

TODO: Write a utility that resets the named trajectory to random values. Use the current fields like bounds and goal to accomplish this.

TODO: Include the ability of objectives to be multiplied by floats.

In [ ]:
trajectory = copy(rob_prob_1.trajectory)
# Need random initial conditions to avoid local minima
# update!(trajectory, :a, random_a_guess(trajectory))
# update!(trajectory, :Ũ⃗, 2 * rand(trajectory.dims[:Ũ⃗], T) .- 1)
parameters = deepcopy(prob.params)

objective = DefaultObjective()
# objective += Objective(parameters[:objective_terms][1])
objective += QuadraticRegularizer(:dda, prob.trajectory, 1e-5)
objective += QuadraticRegularizer(:a, prob.trajectory, 1e-3)

update_bound!(trajectory, :a, Inf)
constraints = trajectory_constraints(trajectory)

ipopt_options = Options()
ipopt_options.hessian_approximation = "limited-memory"

rob_prob_2 = UnitaryRobustnessProblem(
    H_crosstalk,
    trajectory,
    prob.system,
    objective,
    prob.integrators,
    constraints;
    final_fidelity=0.99,
    verbose=false,
    build_trajectory_constraints=false,
    hessian_approximation=true,
    ipopt_options=ipopt_options
)

In [ ]:
solve!(rob_prob_2; max_iter=25)

(i -> println("Fidelity $i"))(unitary_fidelity(rob_prob_2))
(i -> println("Robustness $i"))(infidelity_robustness(H_crosstalk, rob_prob_2))

In [ ]:
plot_control(rob_prob_2)